# Figure 3

COmbine results for all agents and bandits, creating Fig 3 for the draft at [https://www.biorxiv.org/content/10.1101/671362v2]()

See `./informercial/Makefile` for experimental
details.

In [4]:
import os
import numpy as np
import pandas as pd
from pprint import pprint

from IPython.display import Image
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_style('ticks')

matplotlib.rcParams.update({'font.size': 16})
matplotlib.rc('axes', titlesize=16)

from infomercial.exp import meta_bandit
from infomercial.exp import epsilon_bandit
from infomercial.exp import beta_bandit
from infomercial.exp import softbeta_bandit
from infomercial.local_gym import bandit
from infomercial.exp.meta_bandit import load_checkpoint

import gym

# Load and process data

Save results to `.csv` files.

In [5]:
data_path ="/Users/qualia/Code/infomercial/data/"

## BanditOneHigh10

## Example traces

All values for eta

### BanditOneHigh10

In [4]:
env_name = "BanditOneHigh10-v0"
# exp_names = ["exp128",]
exp_names = ["exp194",]
agent_names = ["eta",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result["values_E"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "values_E",                           
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_value_traces_{agent}_{env_name}.csv", index=False)

### BanditOneHigh10

In [9]:
env_name = "BanditOneHigh10-v0"
# exp_names = ["exp128",]
exp_names = ["exp194",]
agent_names = ["eta",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)
    
# -
# exp_names = ["exp131",]
# exp_names = ['exp158',]
exp_names = ['exp206',]
agent_names = ["epsilon",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)
    

# -
# exp_names = ['exp157',]
exp_names = ['exp202',]
agent_names = ["softbeta",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/qualia/Code/infomercial/data/exp206_1.pkl'

### BanditTwoHigh10

In [7]:
env_name = "BanditTwoHigh10-v0"
# exp_names = ["exp132",]
exp_names = ["exp195",]
agent_names = ["eta",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)
    
# -
# exp_names = ["exp131",]
exp_names = ['exp160',]
agent_names = ["epsilon",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)
    

# -
exp_names = ['exp159',]
agent_names = ["softbeta",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)

### BanditHardAndSparse10

In [8]:
env_name = "BanditHardAndSparse10-v0"
# exp_names = ["exp140",]
exp_names = ["exp197",]
agent_names = ["eta",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)
    
# -
exp_names = ['exp164',]
agent_names = ["epsilon",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)

# -
exp_names = ['exp163',]
agent_names = ["softbeta",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/qualia/Code/infomercial/data/exp197_1.pkl'

### BanditUniform121

In [28]:
# exp_names = ["exp136", "exp137", "exp161", "exp162", "exp167", "exp171"]
# agent_names = ["eta", "beta", "softbeta", "epsilon", "anneal-epsilon", "random"]

env_name = "BanditUniform121-v0"

exp_names = ["exp136",]
agent_names = ["eta",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)
    
# -
exp_names = ['exp162',]
agent_names = ["epsilon",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)

# -
exp_names = ['exp161',]
agent_names = ["softbeta",] # , "epsilon"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["values_R"],
            result['regrets'],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
    table = np.vstack(table)
    df = pd.DataFrame(table, 
                      columns=[
                          "episodes", 
                          "values_R", 
                          "regrets",
                          "arms",
                          "exps", 
                          "agents",
                          "n"]
                     )
    df.to_csv(f"../data/example_traces_{agent}_{env_name}.csv", index=False)

# All

## BanditOneHigh10

In [6]:
env_name = "BanditOneHigh10-v0"

# Orginal
# exp_names = ["exp128", "exp129", "exp130", "exp131", "exp149", "exp153"]

# Actor() rerun
# exp_names = ["exp128", "exp129", "exp157", "exp158", "exp165", "exp169"]

# Replicator rerun
exp_names = ["exp194", "exp198", "exp202", "exp206", "exp210", "exp169"]

agent_names = ["eta", "beta", "softbeta", "epsilon", "anneal-epsilon", "random"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["p_bests"], 
            result["values_R"],
            result["actions"],
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "episodes", 
                      "p_bests", 
                      "values_R", 
                      "arms",
                      "exps", 
                      "agents",
                      "n"]
                 )
df.to_csv(f"../data/table_traces_{env_name}.csv", index=False)

# --------------------------------
# Save total regret and reward
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result['total_R'],
            result['p_bests'][-1],
            np.sum(result['regrets']),
            exp_name,
            agent,
            n
        ]

        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "total_R",
                      "p_bests",
                      "regrets", 
                      "exps", 
                      "agents",
                      "n"]
                 )
df.to_csv(f"../data/table_total_{env_name}.csv", index=False)

# --------------------------------
# Save final critic weights/values
critic_names = ["critic_R", "critic", "critic", "critic_R", "critic_R", "critic_R"]

table = []
for agent, exp_name, critic_name in zip(agent_names, exp_names, critic_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))

        critic = result[critic_name]
        rows = [
            list(critic.keys()), # arms
            list(critic.values()),  # values
            [exp_name] * len(critic.keys()),
            [agent] * len(critic.keys()),
            [n] * len(critic.keys())
        ]
        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "arms", 
                      "critic_values", 
                      "exps", 
                      "agents",
                      "n"
                  ]
                 )
df.to_csv(f"../data/table_critic_{env_name}.csv", index=False)

# --------------------------------
# Save bandit p(R)
env = gym.make(env_name)

table = [
    list(range(env.env.n_bandits)),
    env.env.p_dist 
]
table = np.vstack(table).T
df = pd.DataFrame(table, 
                  columns=[
                      "arms", 
                      "p_reward"
                  ]
                 )
df.to_csv(f"../data/table_bandit_{env_name}.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/qualia/Code/infomercial/data/exp198_1.pkl'

In [13]:
result.keys()

dict_keys(['best', 'episodes', 'num_episodes', 'actions', 'p_bests', 'regrets', 'visited_states', 'critic_R', 'total_R', 'scores_R', 'values_R'])

# BanditTwoHigh10

In [14]:
env_name = "BanditTwoHigh10-v0"

# Random search 
# exp_names = ["exp132", "exp133", "exp134", "exp135", "exp150", "exp154"]
# exp_names = ["exp132", "exp133", "exp159", "exp160", "exp166", "exp170"]

# Replicator rerun
exp_names = ["exp195", "exp199", "exp203", "exp207", "exp211", "exp169"]

agent_names = ["eta", "beta", "softbeta", "epsilon", "anneal-epsilon", "random"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result["episodes"],
            result["p_bests"], 
            result["values_R"],
            result["actions"],            
            [exp_name] * len(result["episodes"]),
            [agent] * len(result["episodes"]),
            [n] * len(result["episodes"])
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "episodes", 
                      "p_bests", 
                      "values_R",
                      "arms",
                      "exps", 
                      "agents",
                      "n"]
                 )
df.to_csv(f"../data/table_traces_{env_name}.csv", index=False)

# --------------------------------
# Save total regret and reward
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result['total_R'],
            result['p_bests'][-1],
            np.sum(result['regrets']),
            exp_name,
            agent,
            n
        ]

        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "total_R",
                      "p_bests",
                      "regrets", 
                      "exps", 
                      "agents",
                      "n"]
                 )
df.to_csv(f"../data/table_total_{env_name}.csv", index=False)


# --------------------------------
# Save final critic weights/values
critic_names = ["critic_R", "critic", "critic", "critic_R", "critic_R", "critic_R"]

table = []
for agent, exp_name, critic_name in zip(agent_names, exp_names, critic_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))

        critic = result[critic_name]
        rows = [
            list(critic.keys()), # arms
            list(critic.values()),  # values
            [exp_name] * len(critic.keys()),
            [agent] * len(critic.keys()),
            [n] * len(critic.keys())
        ]
        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "arms", 
                      "critic_values", 
                      "exps", 
                      "agents",
                      "n"
                  ]
                 )
df.to_csv(f"../data/table_critic_{env_name}.csv", index=False)

# --------------------------------
# Save bandit p(R)
env = gym.make(env_name)

table = [
    list(range(env.env.n_bandits)),
    env.env.p_dist 
]
table = np.vstack(table).T
df = pd.DataFrame(table, 
                  columns=[
                      "arms", 
                      "p_reward"
                  ]
                 )
df.to_csv(f"../data/table_bandit_{env_name}.csv", index=False)

# BanditUniform121

In [15]:
env_name = "BanditUniform121-v0"

# Random search
# exp_names = ["exp136", "exp137", "exp138", "exp139", "exp151", "exp155"]
exp_names = ["exp136", "exp137", "exp161", "exp162", "exp167", "exp171"]

# Replicator rerun
exp_names = ["exp196", "exp200", "exp204", "exp208", "exp212", "exp169"]

agent_names = ["eta", "beta", "softbeta", "epsilon", "anneal-epsilon", "random"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        # To much data to save. Keep every Nth point
        Nth = 100
        index = np.arange(0, len(result["episodes"]), Nth)
        
        rows = [
            [result["episodes"][i] for i in index],
            [result["p_bests"][i] for i in index], 
            [result["values_R"][i] for i in index],
            [result["actions"][i] for i in index],
            [exp_name] * len(index),
            [agent] * len(index),
            [n] * len(index)
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "episodes", 
                      "p_bests", 
                      "values_R",
                      "arms", 
                      "exps", 
                      "agents",
                      "n"]
                 )
df.to_csv(f"../data/table_traces_{env_name}.csv", index=False)

# --------------------------------
# Save total regret and reward
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result['total_R'],
            result['p_bests'][-1],
            np.sum(result['regrets']),
            exp_name,
            agent,
            n
        ]

        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "total_R",
                      "p_bests",
                      "regrets", 
                      "exps", 
                      "agents",
                      "n"]
                 )
df.to_csv(f"../data/table_total_{env_name}.csv", index=False)

# --------------------------------
# Save final critic weights/values
critic_names = ["critic_R", "critic", "critic", "critic_R", "critic_R", "critic_R"]

table = []
for agent, exp_name, critic_name in zip(agent_names, exp_names, critic_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))

        critic = result[critic_name]
        rows = [
            list(critic.keys()), # arms
            list(critic.values()),  # values
            [exp_name] * len(critic.keys()),
            [agent] * len(critic.keys()),
            [n] * len(critic.keys())
        ]
        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "arms", 
                      "critic_values", 
                      "exps", 
                      "agents",
                      "n"
                  ]
                 )
df.to_csv(f"../data/table_critic_{env_name}.csv", index=False)

# --------------------------------
# Save bandit p(R)
env = gym.make(env_name)

table = [
    list(range(env.env.n_bandits)),
    env.env.p_dist 
]
table = np.vstack(table).T
df = pd.DataFrame(table, 
                  columns=[
                      "arms", 
                      "p_reward"
                  ]
                 )
df.to_csv(f"../data/table_bandit_{env_name}.csv", index=False)

# BanditHardAndSparse10

In [16]:
env_name = "BanditHardAndSparse10-v0"

# Random search
# exp_names = ["exp140", "exp141", "exp142", "exp143", "exp152", "exp156"]
exp_names = ["exp140", "exp141", "exp163", "exp164", "exp168", "exp172"]

# Replicator rerun
exp_names = ["exp197", "exp201", "exp205", "exp209", "exp213", "exp169"]

agent_names = ["eta", "beta", "softbeta", "epsilon", "anneal-epsilon", "random"]
num_trials = 100

# --------------------------------
# Re-save traces
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        # To much data to save. Keep every Nth point
        Nth = 100
        index = np.arange(0, len(result["episodes"]), Nth)
        
        rows = [
            [result["episodes"][i] for i in index],
            [result["p_bests"][i] for i in index], 
            [result["values_R"][i] for i in index],
            [result["actions"][i] for i in index],            
            [exp_name] * len(index),
            [agent] * len(index),
            [n] * len(index)
        ]
        
        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "episodes", 
                      "p_bests", 
                      "values_R", 
                      "arms",
                      "exps", 
                      "agents",
                      "n"]
                 )
df.to_csv(f"../data/table_traces_{env_name}.csv", index=False)

# --------------------------------
# Save total regret and reward
table = []
for agent, exp_name in zip(agent_names, exp_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))
        
        rows = [
            result['total_R'],
            result['p_bests'][-1],
            np.sum(result['regrets']),
            exp_name,
            agent,
            n
        ]

        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "total_R",
                      "p_bests",
                      "regrets", 
                      "exps", 
                      "agents",
                      "n"]
                 )
df.to_csv(f"../data/table_total_{env_name}.csv", index=False)

# --------------------------------
# Save final critic weights/values
critic_names = ["critic_R", "critic", "critic", "critic_R", "critic_R", "critic_R"]

table = []
for agent, exp_name, critic_name in zip(agent_names, exp_names, critic_names):
    for n in range(num_trials):
        result = load_checkpoint(os.path.join(data_path, f"{exp_name}_{n+1}.pkl"))

        critic = result[critic_name]
        rows = [
            list(critic.keys()), # arms
            list(critic.values()),  # values
            [exp_name] * len(critic.keys()),
            [agent] * len(critic.keys()),
            [n] * len(critic.keys())
        ]
        rows = np.vstack(rows).T
        table.append(rows)
        
table = np.vstack(table)
df = pd.DataFrame(table, 
                  columns=[
                      "arms", 
                      "critic_values", 
                      "exps", 
                      "agents",
                      "n"
                  ]
                 )
df.to_csv(f"../data/table_critic_{env_name}.csv", index=False)

# --------------------------------
# Save bandit p(R)
env = gym.make(env_name)

table = [
    list(range(env.env.n_bandits)),
    env.env.p_dist 
]
table = np.vstack(table).T
df = pd.DataFrame(table, 
                  columns=[
                      "arms", 
                      "p_reward"
                  ]
                 )
df.to_csv(f"../data/table_bandit_{env_name}.csv", index=False)